In [5]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space/gaia/model-tzmvobqh:v0', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [6]:
import torch
from spherinator.models import (
    ConvolutionalDecoder1D,
    ConvolutionalEncoder1D,
    AutoencoderPure,
)

encoder = ConvolutionalEncoder1D(input_dim=344, output_dim=3)
decoder = ConvolutionalDecoder1D(input_dim=3, output_dim=344)
model = AutoencoderPure(encoder=encoder, decoder=decoder)

ckpt = "artifacts/model-tzmvobqh:v0/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
model.load_state_dict(checkpoint["state_dict"])
model.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model, torch.randn(1, 1, 344, device="cpu"), export_options=export_options)
onnx.save("model.onnx")

onnx = torch.onnx.dynamo_export(model.encoder, torch.randn(1, 1, 344, device="cpu"), export_options=export_options)
onnx.save("encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(1, 3, device="cpu"), export_options=export_options)
onnx.save("decoder.onnx")

/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/torch/onnx/_internal/_exporter_legacy.py:116: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/doserbd/.cac